In [6]:
import wandb
import pickle
import os
import tqdm
import numpy as np

def get_sorted_runs(project_name, entity=None, metric="val-losses/mean/pos_l2_final<0.01", run_ids=None, cache_file="wandb_cache.pkl"):
    if os.path.exists(cache_file):
        with open(cache_file, "rb") as f:
            cached_data = pickle.load(f)
    else:
        cached_data = {}
    
    api = wandb.Api()
    runs = api.runs(f"{entity}/{project_name}" if entity else project_name)
    
    run_data = []
    
    for run in tqdm.tqdm(runs):
        if not(len(run.config['tasks']) == 1 and run.config['tasks'][0] == 'stack_blocks'):
            continue
        
        if run.id in cached_data:
            metric_values = cached_data[run.id]
        else:
            metric_values = []
            history = run.scan_history(keys=[metric], page_size=50000,)
            cached_data[run.id] = [row.get(metric, 0) for row in history]
        
        max_metric_value = max(metric_values) if metric_values else 0
        percentile = np.percentile(metric_values, 95) if metric_values else 0
        run_data.append((run, percentile))
    
    with open(cache_file, "wb") as f:
        pickle.dump(cached_data, f)
    
    sorted_runs = sorted(run_data, key=lambda x: x[1], reverse=True)
    
    return sorted_runs

if __name__ == "__main__":
    project = "3d_diffuser_actor_debug"  # Change this
    entity = "felix-herrmann"  # Change this if applicable
    # run_ids = ["toyzo1d5"]  # Define specific run IDs to include
    run_ids = None  # Include all runs
    metric = "val-losses/mean/pos_l2_final<0.01"  # Change this
    
    sorted_results = get_sorted_runs(project, entity, run_ids=run_ids)
    
    for run, value in sorted_results:
        B = run.config['batch_size'] * run.metadata["gpu_count"]
        RN = run.config.get('rot_noise', 0)
        PN = run.config.get('pos_noise', 0)
        PCDN = run.config.get('pcd_noise', 0)
        D = run.config.get('dropout', 0)
        RES = run.config.get('feature_res', 'res3')
        FPS = run.config.get('fps_subsampling_factor', 0)
        HP = run.config.get('gripper_history_as_points', 0)
        CD = run.config.get('use_center_distance', 1)
        CP = run.config.get('use_center_projection', 1)
        VP = run.config.get('use_vector_projection', 1)
        AC = run.config.get('add_center', 1)
        FT = run.config.get('feature_type', 'sinusoid')

        print(f"Max : {value},\t B={B:2}, RN={RN:.2f}, PN={PN:.2f}, PCDN={PCDN:.2f}, D={D}, RES={RES}, FPS={FPS:2}, HP={HP}, CD={CD}, CP={CP}, VP={VP}, AC={AC}, FT={FT}, Run ID: {run.id}, Name: {run.config['name']}")


100%|██████████| 70/70 [00:00<00:00, 272.65it/s]


Max : 0.9123025715351105,	 B=16, RN=0.00, PN=0.00, PCDN=0.00, D=0, RES=res3, FPS= 5, HP=0, CD=1, CP=1, VP=1, AC=1, FT=sinusoid, Run ID: o806fh5g, Name: train_3d_diffuser_actor
Max : 0.8871165037155151,	 B=24, RN=0.00, PN=0.00, PCDN=0.00, D=0, RES=res3, FPS= 1, HP=1, CD=1, CP=1, VP=1, AC=1, FT=sinusoid, Run ID: xyoyswx6, Name: train_point_attention_lang_enhanced
Max : 0.8870987296104431,	 B=24, RN=0.00, PN=0.00, PCDN=0.00, D=0, RES=res3, FPS= 1, HP=1, CD=1, CP=1, VP=1, AC=1, FT=sinusoid, Run ID: dayoc7vs, Name: train_point_attention_lang_enhanced
Max : 0.8869643539190293,	 B=24, RN=0.00, PN=0.00, PCDN=0.00, D=0, RES=res3, FPS= 5, HP=0, CD=1, CP=1, VP=1, AC=1, FT=sinusoid, Run ID: 92ia6mxa, Name: train_3d_diffuser_actor
Max : 0.8827997446060181,	 B=16, RN=0.20, PN=0.00, PCDN=0.00, D=0, RES=res3, FPS= 1, HP=0, CD=1, CP=1, VP=1, AC=1, FT=sinusoid, Run ID: svhqw7gn, Name: train_point_attention_lang_enhanced
Max : 0.8507236242294312,	 B= 8, RN=0.00, PN=0.00, PCDN=0.00, D=0, RES=res3, FPS= 1,